In [ ]:
# Importer les bibliothèques nécessaires
import numpy as np
import pandas as pd
import xlrd
from sklearn.model_selection import  train_test_split
from keras.models import Sequential
from keras.layers import Dense
from colorama import Fore, Style
from sklearn.tree import DecisionTreeClassifier
from plyer import notification

In [ ]:
# Charger les données
Data= pd.read_excel('Women1.xlsx')
#Informations sur les variables
print(Data.info( ) )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11069 entries, 0 to 11068
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Milieu                        11069 non-null  int64  
 1   Age                           11069 non-null  int64  
 2   Niveau_scolaire_plus_atteint  11069 non-null  int64  
 3   Situation_matrimoniale        11069 non-null  int64  
 4   Nbr_fois_marié                11069 non-null  int64  
 5   Travail                       11069 non-null  int64  
 6   Nbr_enfant_vie                11069 non-null  float64
 7   Nbr_fois_consultation         11069 non-null  float64
 8   PF                            11069 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 778.4 KB
None


In [ ]:
# Diviser les données en ensemble d'apprentissage (70%) et ensemble de test (30%)
DataTrain, DataTest = train_test_split(Data, test_size=0.3, random_state=1, stratify=Data.PF)

# Vérification
print(DataTrain.shape)
print(DataTest.shape)

(7748, 9)
(3321, 9)


In [ ]:
# Séparer les variables explicatives de la variable cible

# Y (PF) est la dernière colonne
y_train = DataTrain.iloc [:,-1]
# X (les autres) sont les variables qui précédent la dernière
X_train = DataTrain.iloc [:,:-1]
# De même pour les données du test
y_test = DataTest.iloc [:,-1]
X_test = DataTest.iloc [:,:-1]


# Réseau de neurones

In [ ]:
# Créer le modèle de réseau de neurones
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

# Compiler le modèle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train, y_train, epochs=15, batch_size=32)

# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model.evaluate(X_test, y_test)

Epoch 1/15
243/243 [==============================] - 4s 3ms/step - loss: -3.1433 - accuracy: 0.8353
Epoch 2/15
243/243 [==============================] - 1s 2ms/step - loss: -21.2542 - accuracy: 0.8730
Epoch 3/15
243/243 [==============================] - 1s 3ms/step - loss: -66.2234 - accuracy: 0.8730
Epoch 4/15
243/243 [==============================] - 1s 3ms/step - loss: -140.5401 - accuracy: 0.8730
Epoch 5/15
243/243 [==============================] - 1s 3ms/step - loss: -242.9483 - accuracy: 0.8730
Epoch 6/15
243/243 [==============================] - 1s 3ms/step - loss: -368.5908 - accuracy: 0.8730
Epoch 7/15
243/243 [==============================] - 1s 3ms/step - loss: -515.7115 - accuracy: 0.8730
Epoch 8/15
243/243 [==============================] - 1s 3ms/step - loss: -684.0800 - accuracy: 0.8730
Epoch 9/15
243/243 [==============================] - 1s 3ms/step - loss: -869.8262 - accuracy: 0.8730
Epoch 10/15
243/243 [==============================] - 1s 3ms/step - loss: -1

In [ ]:
# Faire des prédictions avec le modèle entraîné
y_pred = model.predict(X_test)

104/104 [==============================] - 0s 2ms/step


In [ ]:
y_pred

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [ ]:
# Distribution des prédictions
print(np.unique(y_pred, return_counts=True))

(array([1.], dtype=float32), array([3321], dtype=int64))


# Arbre de décision

In [ ]:
# Créer un classificateur d'arbre de décision
clf = DecisionTreeClassifier()

# Adapter le classificateur aux données d'entraînement
clf.fit(X_train, y_train)

# Prédire les étiquettes pour l'ensemble de test
y_pred1 = clf.predict(X_test)

In [ ]:
# Distribution des prédictions
print(np.unique(y_pred1, return_counts=True))

(array([1, 2], dtype=int64), array([3061,  260], dtype=int64))


In [ ]:
# Afficher des messages de détection ou de non-détection d'un risque humain

for i in range(len(y_pred1)):
    if y_pred1[i] == 1:
        print(Fore.GREEN + "Non Detection d'un risque humain" + Style.RESET_ALL)
    else:
        print(Fore.RED + "Detection d'un risque humain" + Style.RESET_ALL)

Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Detection d'un risque humain
Non Detection d'un risque humain
Detection d'un risque humain
Non Detection d'un risque humain
Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque humain
Non Detection d'un risque huma

In [ ]:
# Programme d'alerte sous forme de notification

if 2 in y_pred1 :
    index=np.where(y_pred1==2)[0][0]+1
    notification.notify(
        title='Alerte Risque',
        message=f'Un risque a été détecté à l\'emplacement {index}',
        app_name='SOS',
        timeout=5
    )